In [1]:
!nvidia-smi

Fri May 16 08:54:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install svgwrite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 1.8 MB/s eta 0:00:00


**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [3]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


## Install Segment Anything Model (SAM) and other dependencies

In [4]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

  Preparing metadata (setup.py) ... done


In [5]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision==0.23.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 842.5/842.5 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.9 MB/s eta 0:00:00


In [6]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import supervision as sv
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import svgwrite
from skimage import measure

### Download SAM weights

In [7]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [8]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [9]:
CHECKPOINT_PATH = "/content/weights/sam_vit_h_4b8939.pth"
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [10]:
mask_generator = SamAutomaticMaskGenerator(sam)

# Output format

`SamAutomaticMaskGenerator` returns a `list` of masks, where each mask is a `dict` containing various information about the mask:

* `segmentation` - `[np.ndarray]` - the mask with `(W, H)` shape, and `bool` type
* `area` - `[int]` - the area of the mask in pixels
* `bbox` - `[List[int]]` - the boundary box of the mask in `xywh` format
* `predicted_iou` - `[float]` - the model's own prediction for the quality of the mask
* `point_coords` - `[List[List[float]]]` - the sampled input point that generated this mask
* `stability_score` - `[float]` - an additional measure of mask quality
* `crop_box` - `List[int]` - the crop of the image used to generate this mask in `xywh` format

# Create a helper function to convert the segmentation masks into SVG images

In [11]:
# Create an helper function to convert the segmentation masks into svg images
def boolean_masks_to_single_svg(masks, output_filepath):

    # 1. Create a single SVG drawing object.  This will hold all the paths.
    dwg = svgwrite.Drawing(output_filepath, profile='tiny')

    # 2. Iterate through each boolean mask in the input list.
    for i, mask in enumerate(masks):
        # 3. Find the contours of the current mask.
        #    - Convert the boolean mask to integers (True -> 1, False -> 0)
        #    - Use a level of 0.5 to find the boundary between 0 and 1.
        contours = measure.find_contours(mask.astype(int), 0.5)

        # 4. Iterate through each contour found for the current mask.
        #    A mask might have multiple disconnected parts, each with its own contour.
        for contour in contours:
            # 5. Build the SVG path data string.
            #    - Start with "M" (move to) the first point of the contour.  Note the (x, y) coordinate swap.
            path_data = f"M{contour[0, 1]},{contour[0, 0]}"
            #    - Add "L" (line to) commands for the remaining points.
            for point in contour[1:]:
                path_data += f" L{point[1]},{point[0]}"
            #    - Close the path with "Z" (draw a line back to the start).
            path_data += " Z"

            # 6. Create an SVG path element.
            #    - Use 'none' for the fill color, so only the outline is visible.
            #    - Set the stroke color and width.  You can customize these!
            #    -  Example of using different stroke colors for different masks
            stroke_colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown',
                             'cyan', 'magenta', 'yellow', 'lime', 'teal', 'olive']
            stroke_color = stroke_colors[i % len(stroke_colors)]  # Cycle through colors

            path = dwg.path(d=path_data, fill='none', stroke=stroke_color, stroke_width=1)

            # 7. Add the path element to the SVG drawing.
            dwg.add(path)

    # 8. Save the complete SVG drawing to the output file.
    dwg.save()
    print(f"All mask paths saved to a single SVG file: {output_filepath}")

In [12]:
def sam_mask_svg_generation(image_path):
  image_bgr = cv2.imread(image_path)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  sam_result = mask_generator.generate(image_rgb)
  mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)
  detections = sv.Detections.from_sam(sam_result=sam_result)
  annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)
  masks = [mask['segmentation'] for mask in sorted(sam_result, key=lambda x: x['area'], reverse=True)]
  boolean_masks_to_single_svg(masks, "SegmentationMask.svg")

In [ ]:
sam_mask_svg_generation("/content/996159.jpg")

All mask paths saved to a single SVG file: SegmentationMask.svg


In [ ]:
# # To get coordinates for each mask (bounding polygons)
# for i, mask in enumerate(mask):
#     # Convert mask to polygons
#     polygons = sv.mask_to_polygons(mask)

#     print(f"Mask {i} coordinates:")
#     for polygon in polygons:
#         print(polygon)  # Each polygon is an array of (x,y) points

In [ ]:
# # From sam_result
# for i, mask_data in enumerate(sam_result):
#     mask = mask_data['segmentation']  # This is the boolean mask

#     # Get polygons from the mask
#     polygons = sv.mask_to_polygons(mask)

#     print(f"Mask {i} coordinates:")
#     for polygon in polygons:
#         print(polygon)